In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

| | |
|-|-|
|Author(s) | [Lavi Nigam](https://github.com/lavinigam-gcp) |


**bit.ly / io24-gemini-api**

# Step 1 - Google Cloud Vertex AI Gemini API

[What Changed: ]
- SDK installation

Before:

- ! pip install --upgrade google-generativeai

After:

- ! pip install --upgrade google-cloud-aiplatform

In [1]:
# Library installation # needs restarts

# ! pip install --upgrade google-generativeai
! pip install --upgrade google-cloud-aiplatform
! pip install PyPDF2
# ! apt-get install poppler-utils
# ! pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.48.0
    Uninstalling google-cloud-aiplatform-1.48.0:
      Successfully uninstalled google-cloud-aiplatform-1.48.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00


[What Changed: ]
- Authentication  

Before:

- genai.configure(api_key="API Key")

After:

 - Authenticate Colab user to Google Cloud
 - Vertex AI init with Project ID and Region

  [Not required in Colab Enterprise]

In [1]:
# Authentication

# genai.configure(api_key="")

import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = ""  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
# Downloading data from Google Drive
!gdown --folder https://drive.google.com/drive/folders/1RfMScB0Y1LUQdW5tvjyYA4_D21H5HMaT?usp=sharing -O /content/data/

Retrieving folder contents
Processing file 1iT0qe9EU8RQyiAlrMlJ-SH-E_WtMGFNm goog-10-q-q2-2023-4-1-15.pdf
Processing file 1qs4blSxatemVxpZ3dt1yzESDa4aNWptD Google's newest and most capable AI  Gemini.mp3
Processing file 1oX_TaQZjOiuDUwsTrqz7zq5HMEe7cEMR Google's newest and most capable AI _ Gemini.mp4
Processing file 1totf67pJ3XYhILCY0TrbS1tHm7PwCZES Responsible AI _ Google I_O 2023.mp4
Processing file 1kaq58S4oRmuZgDqK109uc6mwedVmPAby stage_1_prototype_Alphabet 2023 Q4 Earnings Call.mp3
Processing file 1GXGfwoe8-vsoOM1E8SBOCybc7R5XRd2P stage_1_prototype_Google Cloud TPU blog.pdf
Processing file 19aqpEZqTVTERbySYVrtDgSjuUBUabeSE What's next for AI and Google Search _ Google I_O 2023.mp4
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1iT0qe9EU8RQyiAlrMlJ-SH-E_WtMGFNm
To: /content/data/Idea/goog-10-q-q2-2023-4-1-15.pdf
100% 217k/217k [00:00<00:00, 96.1MB/s]
Downloading...
From: h

[What Changed: ]

-


Before:

-

After:

-


[What Changed: ]

- Import statement


Before:

- import google.generativeai as genai

After:

- import vertexai.generative_models as genai


In [5]:
# Library
from IPython.display import display, Markdown, Latex
# import google.generativeai as genai
import vertexai.generative_models as genai
import PyPDF2
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown

[No Change ]

- Model Config and Safety Setting


Before:



After:


In [12]:
# Set up the model
from vertexai.generative_models import (
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold,
)


generation_config = GenerationConfig(
    temperature=1,
    max_output_tokens=8192,
)


safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

model = genai.GenerativeModel(model_name="gemini-1.5-pro-preview-0409",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

model_flash = genai.GenerativeModel(model_name="gemini-1.5-flash-preview-0514",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

### PDF

[No Change]

In [7]:
#@title Helper Functions
## Helper Function

def pdf_to_dict_str(file_path):
    """Reads a PDF file and returns a dictionary with page numbers as keys and page text as values."""

    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        page_dict = {}
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            page_dict[page_num + 1] = page_text  # Page numbers start from 1

        return str(page_dict)

In [19]:
%%time

prompt_parts = [
  pdf_to_dict_str("/content/data/Idea/stage_1_prototype_Google Cloud TPU blog.pdf"),
  "What are key achievement for Google Cloud from the following blog. Mention in bullet.",
]
response = model.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 331 ms, sys: 4.12 ms, total: 335 ms
Wall time: 6.17 s


Here are the key achievements of Google Cloud based on the information you provided:                               

 • Google Cloud TPU ran the world’s largest distributed training job for LLMs across 50,000+ TPU v5e chips.        
 • The training was performed using the Cloud TPU Multislice Training on a compute cluster of 50,944 Cloud TPU v5e 
   chips.                                                                                                          
 • The cluster used has more AI accelerators than the TOP1 Supercomputer Frontier at Oak Ridge National Laboratory,
   which featured 37,888 AMD MI250X GPUs.                                                                          
 • The distributed training job was set up on the JAX framework, utilizing XPK, GKE, MaxText, AQT, and other       
   components of the JAX training stack.                                                                           
 • Achieved as high as 66.86% MFU with BF16 training on a single TPU v5e pod and strong scaling outcomes when      
   expanding to 160 pods.                                                                                          
 • Achieved an impressive observed 5.32 exa-OP/s with INT8 quantized training.                                     
 • Open-sourced all the code used in the project.

In [18]:
%%time

response = model_flash.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 25.5 ms, sys: 2.32 ms, total: 27.8 ms
Wall time: 3.31 s


The provided document doesn't provide enough information to answer those questions comprehensively. Here's why:    

 • Focus on Overall Financial Performance: The document primarily focuses on Alphabet's overall financial          
   performance, including consolidated statements and notes, as well as disclosures regarding revenue sources and  
   accounting policies.                                                                                            
 • Limited Google Cloud Specific Data:  While it does break down revenue by category, the information on Google    
   Cloud is limited to revenue figures for the last two quarters. It doesn't offer historical performance data,    
   detailed insights into the factors affecting performance, or direct comparisons to competitors.                 
 • Lack of Management Commentary: The document lacks any management commentary or discussion of Google Cloud's     
   specific strategies, challenges, or goals.                                                                      

To answer your questions more thoroughly, you'd need to consult additional sources:                                

 • Alphabet's Annual Reports (Form 10-K): These reports provide a more detailed analysis of Google Cloud, including
   historical performance data, explanations of key performance drivers, and management's outlook on the business. 
 • Earnings Call Transcripts:  These transcripts contain detailed discussions from Alphabet's leadership team about
   the company's performance, including Google Cloud. They often provide valuable insights into the business's     
   direction and challenges.                                                                                       
 • Industry Analyst Reports:  Industry analysts closely follow the cloud computing market and publish reports      
   analyzing the performance of Google Cloud and its competitors.                                                  

In general:                                                                                                        

 • Google Cloud has been growing significantly in recent years. It is a key focus area for Alphabet as they seek to
   diversify revenue away from advertising.                                                                        
 • Google Cloud faces strong competition from companies like Amazon Web Services (AWS) and Microsoft Azure.        

To get specific details on Google Cloud's performance, strategies, and competitive position, it's essential to     
consult those additional sources.

In [34]:
%%time

prompt_parts = [
  pdf_to_dict_str('/content/data/Idea/goog-10-q-q2-2023-4-1-15.pdf'),
  """Answer based on the document provided:
- How has Google Cloud performed in last quarters?
- What factors have influenced its numebrs and is it positive or negative?
- What is the leadership view on Google Cloud business?
- What are some changes and decisions they have made along the way to achieve their goals?
- How are they doing with respect to other cloud companies?
  """,
]
response = model.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 741 ms, sys: 0 ns, total: 741 ms
Wall time: 7.94 s


The provided text is the Alphabet Inc. 10-Q report for the quarterly period ended June 30, 2023. This document does
not contain information about the market share of Google Cloud compared to its competitors, the leadership view on 
the business, or the changes and decisions made.                                                                   

However, the document does state that:                                                                             

 • Google Cloud revenues were $6.276 billion for the three months ended June 30, 2022 and $8.031 billion for the   
   three months ended June 30, 2023.                                                                               
 • Google Cloud revenues were $12.097 billion for the six months ended June 30, 2022 and $15.485 billion for the   
   six months ended June 30, 2023.                                                                                 

This shows that Google Cloud has grown its revenue in the past year. The specific factors that influenced these    
numbers are not provided in this document.                                                                         

To find out more about the leadership view, strategic decisions, and market share, you would need to consult other 
sources like:                                                                                                      

 • Alphabet Inc.'s  Annual Report (Form 10-K) for a more comprehensive view of the business.                       
 • Earnings call transcripts and presentations: These often provide qualitative insights from leadership.          
 • Industry analyst reports: These offer comparative analyses of Google Cloud's performance against competitors.

In [33]:
%%time

response = model_flash.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 40.9 ms, sys: 4.59 ms, total: 45.5 ms
Wall time: 5.06 s


The provided document doesn't offer specific details on Google Cloud's performance compared to other cloud         
companies or provide insights into leadership's views on the business.  However, it does give us some data points  
to understand its financial performance:                                                                           

 • Revenue growth: Google Cloud revenue increased from $6,276 million in the three months ended June 30, 2022, to  
   $8,031 million in the same period of 2023.  The trend continues for the six-month period with revenue going from
   $12,097 million to $15,485 million.  This indicates strong revenue growth for the business.                     
 • Revenue Backlog:  As of June 30, 2023, Google Cloud had a revenue backlog of $60.6 billion, representing        
   commitments for future services. This suggests a significant pipeline of future revenue.                        
 • Deferred revenue:  The document mentions deferred revenues primarily relate to Google Cloud and Google other. As
   of December 31, 2022, total deferred revenue was $4.5 billion, of which $2.1 billion was recognized as revenue  
   during the six months ended June 30, 2023. This indicates a substantial portion of Google Cloud revenue is      
   recognized over time as services are delivered.                                                                 

Factors influencing Google Cloud numbers:                                                                          

While the document doesn't directly state factors influencing Google Cloud's performance, it mentions several      
trends that could be relevant:                                                                                     

 • Shift from offline to online: The document states that the continuing shift from offline to online will likely  
   benefit the company's business. This suggests that Google Cloud could be benefiting from the growing adoption of
   cloud services in various sectors.                                                                              
 • Investment in technical infrastructure: Alphabet continues to invest in its technical infrastructure, which     
   could be driving the growth of Google Cloud services.                                                           
 • Acquisition and strategic investments: Alphabet's strategy of acquiring and investing in new businesses,        
   products, services, and technologies could be contributing to Google Cloud's offerings and growth.              

To get a more complete picture of Google Cloud's performance and the leadership's views, you would need to access: 

 • Alphabet's 10-K report: This annual report provides a more comprehensive analysis of Google Cloud's performance,
   including comparisons to competitors, detailed risk factors, and the company's strategy for the business.       
 • Earnings call transcripts: During earnings calls, Alphabet executives discuss Google Cloud's performance in     
   detail, offering insights into their strategies, challenges, and future plans.                                  
 • Industry research reports:  Research reports from firms specializing in the cloud computing industry provide    
   independent analysis and comparisons of Google Cloud to its competitors.

### Audio File

[What Changed: ]

- File Upload method


Before:

- genai.upload_file

After:

- Part.from_uri [reading from GCS]


In [21]:
from vertexai.generative_models import Part

In [22]:
# earning_calls = genai.upload_file(path="/content/stage_1_prototype_Alphabet 2023 Q4 Earnings Call.mp3",
#                             display_name="earning_calls")

earning_calls = Part.from_uri(uri = "gs://gemini-lavi-asset/production/earning_calls/Alphabet 2023 Q4 Earnings Call.mp3",
    mime_type="audio/mpeg")

In [23]:
%%time

prompt_parts = [
  earning_calls,
  "How has performance max worked out for Google Ads?",
]
response = model.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 255 ms, sys: 32.7 ms, total: 288 ms
Wall time: 48.8 s


Performance Max has remained a bright spot for Google Ads in Q4 of 2023 and drove incremental query growth during  
key shopping moments like Cyber Five. Google is seeing strong feedback on automatically created assets for         
Performance Max in Q4 and they are expanding to eight languages. It is also helping retailers accelerate omni      
growth and capture holiday demand.

In [25]:
%%time

response = model_flash.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 100 ms, sys: 16.8 ms, total: 117 ms
Wall time: 18.9 s


According to the conference call, performance max has been a bright spot for Google Ads. The team is very excited  
about the demand gen momentum. They have tens of thousands of advertisers testing it and seeing an average of 6%   
more conversions per dollar versus image-only ads in discovery campaigns.                                          

Performance max has also been a win for retailers looking to accelerate omni-growth and capture holiday demand.

### Images

[What Changed: ]

- File Upload method
- PDF to Image method


Before:

- genai.upload_file
- pdf_to_images

After:

- Part.from_uri [it can read pdf as image directly]
- No need of custom function


In [26]:
blog_pdf = Part.from_uri(uri = "gs://gemini-lavi-asset/idea_doc/stage_1_prototype_Google Cloud TPU blog.pdf",
    mime_type="application/pdf")

In [27]:
%%time

prompt_parts = [
  blog_pdf,
  "what is the emfu for bf16 and 128b parameter model with 1 tpu v5e pod? Cite the table and page number and explain the significance of the results",
]
response = model.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 37.5 ms, sys: 7.18 ms, total: 44.7 ms
Wall time: 6.67 s


According to the table "MaxText LLM Training Results" on page 14, the EMFU for a BF16 128B parameter model with 1  
TPU v5e pod is 56.06%.                                                                                             

This result signifies that the model training achieved 56.06% of the theoretical peak performance of the TPU v5e   
hardware for floating-point operations. EMFU (Effective Model FLOPs Utilization) considers both quantized and      
floating-point operations, providing a broader view of hardware utilization efficiency. While a higher EMFU is     
generally desirable, achieving over 50% utilization on a large language model with a single TPU v5e pod            
demonstrates efficient use of the hardware's capabilities.

In [28]:
%%time

response = model_flash.generate_content(prompt_parts)
rich_Markdown(response.text)

CPU times: user 24.2 ms, sys: 4.14 ms, total: 28.3 ms
Wall time: 3.55 s


The EMFU for the BF16 and 128B parameter model with 1 TPU v5e pod is 56.06%. This can be found in the table on page
14 of the document.                                                                                                

This result is significant because it shows that the model is achieving a good level of efficiency even when       
running on a single pod. The EMFU is a measure of how well the model is utilizing the available compute resources. 
A higher EMFU indicates that the model is using more of the available compute power, which can lead to faster      
training times.                                                                                                    

In this case, the EMFU of 56.06% suggests that the model is utilizing roughly half of the available compute power. 
This is a good result for a single pod, and it suggests that the model could be even more efficient if it were run 
on a larger cluster of pods.

### Video

[What Changed: ]

- File Upload method
- Splitting video into frame

Before:

- genai.upload_file
- process_video

After:

- Part.from_uri [reading from GCS]
- No need to convert the video->frames


In [ ]:
#@title Helper Functions
## Helper Function

import cv2
import os
import shutil

def process_video(video_file_path, full_video=False):
    """
    Extracts frames from a video, optionally uploads them, and returns a list of File objects.

    Args:
        video_file_path (str): Path to the video file.
        full_video (bool, optional): If True, uploads all frames. Otherwise, uploads a 10-second slice. Defaults to False.

    Returns:
        list: A list of File objects, each containing file path, response (if uploaded), and timestamp.
    """

    FRAME_EXTRACTION_DIRECTORY = "/content/frames"
    FRAME_PREFIX = "_frame"

    def create_frame_output_dir(output_dir):
        """Creates or cleans up the frame output directory."""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        else:
            shutil.rmtree(output_dir)
            os.makedirs(output_dir)

    def get_timestamp(filename):
        """Extracts the timestamp from a frame filename."""
        parts = filename.split(FRAME_PREFIX)
        if len(parts) != 2:
            return None  # Incorrect format
        return parts[1].split('.')[0]

    class File:
        """Represents a file with path, display name, timestamp, and optional response."""
        def __init__(self, file_path, display_name=None):
            self.file_path = file_path
            self.display_name = display_name or os.path.basename(file_path)
            self.timestamp = get_timestamp(file_path)

        def set_file_response(self, response):
            self.response = response

    # Extract frames
    create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
    vidcap = cv2.VideoCapture(video_file_path)
    if not vidcap.isOpened():
        raise ValueError("Could not open video file.")

    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_duration = 1 / fps
    output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
    frame_count = 0
    count = 0

    while vidcap.isOpened():
        success, frame = vidcap.read()
        if not success:
            break  # End of video

        if int(count / fps) == frame_count:
            min = frame_count // 60
            sec = frame_count % 60
            time_string = f"{min:02d}:{sec:02d}"
            image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
            output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
            cv2.imwrite(output_filename, frame)
            frame_count += 1

        count += 1

    vidcap.release()
    print(f"Extracted: {frame_count} frames")

    # Process and optionally upload frames
    files = [File(os.path.join(FRAME_EXTRACTION_DIRECTORY, file)) for file in sorted(os.listdir(FRAME_EXTRACTION_DIRECTORY))]
    files_to_upload = files if full_video else files[40:50]  # Adjust slice as needed

    uploaded_files = []
    print(f'Uploading {len(files_to_upload)} files...')
    for file in files_to_upload:
        print(f'Uploading: {file.file_path}...')
        try:
            response = genai.upload_file(path=file.file_path)  # Assuming 'genai' is available
            file.set_file_response(response)
            uploaded_files.append(file)
        except Exception as e:
            print(f"Error uploading {file.file_path}: {e}")

    print(f"Uploaded: {len(uploaded_files)} files")
    return uploaded_files

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

In [ ]:
# # Upload a video ~ 1min
# video_file_name = "/content/What's next for AI and Google Search _ Google I_O 2023.mp4"
# processed_files_search = process_video(video_file_name, full_video=True)

In [29]:
video = Part.from_uri(uri = "gs://gemini-lavi-asset/idea_doc/What's next for AI and Google Search _ Google I_O 2023.mp4",
    mime_type="video/mp4")

In [30]:
%%time

prompt_parts = [
  video,
  "Describe this video and How is google using generative ai in search? Give response in bullet",
]
response = model.generate_content(prompt_parts)
Markdown(response.text)

CPU times: user 62.2 ms, sys: 9.71 ms, total: 71.9 ms
Wall time: 11.8 s


Here's a breakdown of the video and how Google is using generative AI in search:

**Video Description:**

* The video showcases Google's new AI-powered search features.
* Users can ask complex questions in a single search bar, including specific requirements and filters.
* Google's generative AI provides conversational, summarized answers drawing from various web sources.
* It goes beyond traditional search results by:
    * Offering product suggestions with visual examples.
    * Creating personalized training plans.
    * Comparing options side-by-side.
    * Engaging in debates on popular topics.

**How Google Uses Generative AI in Search:**

* **Understanding complex queries:** AI allows Google to grasp the nuances and multiple facets of a user's question, even if it's phrased conversationally.
* **Synthesizing information:**  Instead of just providing links, generative AI assembles relevant information from various sources to create a concise summary directly in the search results.
* **Enhancing search results:**  AI goes beyond text by integrating visuals like product images and videos, making results more engaging and informative.
* **Interactive follow-ups:** The "ask a follow-up" button allows users to dive deeper into a topic and refine their search, creating a more dynamic search experience. 
* **Personalization:** AI tailors responses based on user preferences and search history, offering more relevant and helpful information.


In [31]:
%%time
response = model_flash.generate_content(prompt_parts)
Markdown(response.text)

CPU times: user 40.3 ms, sys: 7.41 ms, total: 47.7 ms
Wall time: 7.19 s


Sure, here is a description of the video and how Google is using Generative AI in search:

**Video Description:**
The video showcases Google's new Search experience, which is powered by Generative AI. 
The video highlights the following:

* **Evolution of Search:**  The video starts by stating that AI has been behind the evolution of search for over a decade and now, it's moving front and center.
* **AI-Powered Search:**  It demonstrates how Generative AI can understand complex search queries and provide comprehensive results. 
* **Interactive and Conversational:**  The video shows how users can interact with search results, ask follow-up questions, and receive more relevant information.
* **Personalized Results:**  The video emphasizes that search results are tailored to individual needs and preferences.
* **Unlocking New Search Possibilities:** The video concludes by highlighting that Generative AI is unlocking new ways to search and that Google is committed to continuously improving the Search experience.

**How Google is Using Generative AI in Search:**

* **Understanding Complex Queries:**  Generative AI helps Google understand complex and nuanced search queries that may not be easily expressed with traditional keywords.
* **Generating Comprehensive Results:**  AI can generate summaries of information from various sources, providing users with a broader understanding of the topic.
* **Facilitating Conversation:**  Generative AI enables users to have a more conversational experience with Search, asking follow-up questions and receiving deeper insights.
* **Personalization:**  AI can personalize search results based on user history, interests, and location.
* **Uncovering Hidden Information:**  Generative AI can help Google identify and present relevant information that may not be easily found through traditional keyword searches.

In essence, the video shows how Google is leveraging the power of Generative AI to enhance the Search experience, making it more intuitive, comprehensive, and personalized for users. 
